# Exploration 7. 머신러닝 모델을 제품으로 만들어보자 : MLOps 기초
**MLOps 이론 공부**

## 학습 목표 및 내용
- 지금까지 만들었던 딥러닝 모델을 KerasTuner로 하이퍼파라미터 튜닝을 한 뒤에 TFServing으로 API, tflite 파일 생성
- 머신러닝 모델을 실험실에서만 진행했을때 생기는 문제점
- MLOps의 정의, ML 시스템의 구성요소
- TFX (TensorFlow Extended) 
     - **Keras Tuner, TensorFlow Serving**
- KerasTuner를 활용해서 **하이퍼 파라미터 튜닝**
- TensorFLow Serving과 TFLite로 **모델 배포**



# TFX - Keras Tuner
- 하이퍼파라미터 튜닝
- keras의 MNIST 데이터셋 사용

In [ ]:
# # install KerasTuner 
# !mkdir ~/aiffel/mlops
# !pip install keras-tuner

In [ ]:
# import library
import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os

In [ ]:
# load dataset from keras
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
# add extra dimension for CNN 
X_train = x_train.reshape(-1,28, 28, 1) 
X_test = x_test.reshape(-1,28,28,1)

# label encoding - categorical 
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
# split data - train, validation = 8:2 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
print(X_train.shape)

## DeepTuner
- `kerastuner.Tuner`를 인자로 하는 class
- `run_trial`,`save_model`,`load_model` 함수 실행
    - `run_trial`에서 중요한 부분! : `hypermodel`, `trial`
  
  
**KerasTuner의 hypermodel**

- 모델 공유 및 재사용하기 위해 검색 공간을 캡슐화하는 모델
- `hp` 인수를 사용해서 `keras.Model` 생성
    - 만들고 싶은 모델을 쌓는 과정에서 하이퍼파라미터 튜닝을 위한 검색공간을 만들때 hp 인수를 사용해서 모델 생성
- `build` 메소드를 활용하면 **모델빌드 + 하이퍼파라미텨 튜닝 시작**


**KerasTuner의 trial**
- Oracle에 속하는 class
    - Oracle : KerasTuner`의 모든 검색 알고리즘에서 사용하는 기본 클래스
- 종류 : RandomSearchOracle, BayesianOptimizationOracle, HyperbandOracle
    - KerasTuner가 하이퍼파라미터를 정할때 사용하는 알고리즘
- `trial.hyperparameter` : Oracle이 찾아야하는 하이퍼파라미터
    - = `hypermodel`의 `hp`


In [ ]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)      # 모델빌드 + 하이퍼파라미터 튜닝 시작
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(  # trial : batch size 검색
            'batch_size', [16, 32]), **fit_kwargs)


        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}

## Build model
- hypermodel 생성
- `hp` 변수로 하이퍼파라미터 튜닝

In [ ]:
def build_model(hp):
    model = tf.keras.Sequential()
    
    model.add(tf.keras.Input(shape = X_train.shape[1:], name = 'inputs'))  # hypermodel은 input 지정 필수
    
    # Con2D layer 개수 검색 (1~10개)
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):       
              model.add(tf.keras.layers.Conv2D(hp.Int(         # conv2d units 검색 (32~128 범위)           
                  'units_{i}'.format(i=i), min_value=32, max_value=128, step=5), (3,3),activation='relu'))
            
    model.add(tf.keras.layers.Flatten())
    
    # Dense layer 개수 검색 (1~3개)
    for i in range(hp.Int('n_connections', 1, 3)):
        model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes',        # dense units 검색 (32, 64, 128, 256 중 선택)
                                  values=[32,64,128, 256]), activation = 'relu'))   
        
    model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))
    
    model.compile(optimizer = 'adam',
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

## KerasTuner 정의 & 파라미터 탐색
- BayesianOptimizationOracle을 사용
    - Objective :accuracy, max
    - trial : 10
- `search` : 파라미터 탐색 함수
- 무거운 모델을 돌릴경우 하이퍼파라미터 튜닝 작업 소요시간이 늘어남
    - **search epoch를 3-4로 작게 설정 -> 최고의 하이퍼파라미터 추출 -> 본격 모델학습때 epoch를 넉넉하게 설정**

In [ ]:
# make keras tuner
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='my_keras_tuner')

# Navigate parameters
my_keras_tuner.search(
    X_train, y_train, validation_data=(X_val, y_val), epochs=3)

## Select best hyperparameter
- `KerasTuner.get_best_hyperparamters`를 이용해서 가장 좋은 하이퍼파라미터 추출
- 추출한 하이퍼파라미터를 build_model()에 넣어서 모델 생성

In [ ]:
# Extract the best hyperparameters
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]

# Make the best model with the hyperparameters
model = build_model(best_hps)
model.summary()

## Train & Evaluate model
- 위에서 만든 모델로 학습후 모델평가

In [ ]:
# Train the best model
model.fit(X_train, y_train, batch_size=32, epochs = 5)

In [ ]:
# Evaluate the model
model.evaluate(X_test, y_test)

## Save model
- HDF5(.h5) 저장방식은 TensorFlow나 Keras에서 선호하지않는 방식
- TensorFlow 공식 지원하는 모델저장방식은 'SavedModel'

**TensorFlow SavedModel**
- .h5파일처럼 가중치와 모델을 전부 하나의 파일로 관리X
- **모델, 가중치를 따로 구분해서 저장 => 모델을 배포할때 유리**
- 구성
    - **saved_model.pb** : pb는 프로토콜 버퍼를 의미, 해당 파일은 내보낸 모델 그래프 구조를 포함
    - **variables** : 내보낸 변수값이 있는 이진 파일, 내보낸 모델 그래프에 해당하는 체크포인트를 포함
    - **assets** : 내보낸 모델을 불러올 때 추가적인 파일이 필요한 경우 이 폴더에 파일이 생성됨

- Keras '.keras'파일 : .h5파일과 마찬가지로 가중치와 모델을 전부 하나의 파일로 관리

In [ ]:
save_path = os.getenv('HOME') + '/aiffel/mlops/best_model/1'
fname = os.path.join(save_path, 'model')
model.save(fname)

---

# 모델 배포
- 배포 방식 :
    - 클라우드 활용 배포 : TFServing
    - 경량화된 모델생성후 휴대폰 같은 디바이스에서도 모델이 실행되도록 배포 : TFLite
    

---

# TFServing 
- 텐서플로우 그래프 배포, 표준화된 엔드포인트 제공
- 모델 및 버전관리, 정책 기반으로 모델 서비스 가능
- 지연 시간을 최대한 짧게 만드는 고성능 처리량에 초점을 둔 방식

- 방식:
    - Docker를 활용한 배포
    - 우분투 터미널을 활용한 배포
        - 주의! 우분투 터미널 실습의 경우 실제 결과물이 나오려면 로컬에서 진행해야 함
        - LMS 시스템에 들어가 있는 GPU클라우드는 Docker Image이며 쿠버네티스로 관리중
        - **WSL2**와 **Docker 환경세팅** 필요!
            - [WSL2 설치 + 윈도우에서 Docker 설치하기](https://axce.tistory.com/121)
            - [파일을 우분투 디렉토리로 옮기는 방법](https://bbeomgeun.tistory.com/139)
            - [macOS에서 Docker 설치하기](https://kplog.tistory.com/288)
            
## TFServing 우분투 터미널 실습
- 우분투에 tensorflow-model-server를 설치해 배포용 텐서플로우 서버를 구축 script
```shell
echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -
```
```shell
sudo apt update
```
```shell
sudo apt install tensorflow-model-server
```


- 모델 배포 스크립트

```shell
tensorflow_model_server --port=8500 \
						 --rest_api_port=8501 \
						 --model_name=my_model \
						 --model_base_path=/aiffel/mlops/best_model/1/model 
```
- `model_base_path`는 **SavedModel**이 있는 디렉토리주소
    - 주의 : SavedModel넣을때 model 디렉토리 내부에 **숫자 '1' 폴더**를 안에 넣어야함!(?)
    
    
## TFServing Docker 실습
- 도커 설치후 
```shell
docker pull tensorflow/serving
```

- WSL2 shell(mac은 터미널)에 아래 명령어 실행
```shell
docker run -p 8500:8500 \
			-p 8501:8501 \
			--mount type=bind, source=/tmp/models, target=/models/my_model
			-e MODEL_NAME=my_model \
			-e MODEL_BASE_PATH=/models/my_model \
			-t tensorflow/serving
```
    - 첫번째 줄 : 기본포트 지정
    - 2번째줄 : API port
    - 3번째줄 : 모델 디렉토리 마운트
    - 4번째줄 : 모델 이름 지정
    - 5번째줄 : 모델의 기본경로
    - 마지막줄 : tensorflow/serving을 사용하겠다!
    

---
# TFLite 경량화 모델 생성
- TensorFlow로 만들어진 모델을 휴대폰같은 기기에서도 실행될수 있게 더 작은 모델 크기로 변환해서 배포하는데 사용하게 만드는 방법
- TFLite의 경우 양자화라는 기법을 활용해 모델의 크기를 줄임 -> 모델의 성능이 크게 저하되지 않음
- TensorFlow에 내장되어 있어 별도의 설치가 없이 작동하는 방식
- `tf.lite.TFLiteConverter`메소드 활용

*현재 LMS에서 tflite모델이 만들어지긴 하지만, 원인을 모르겠으나 모바일에서 tflite파일을 구동할때 중요한 '서명'이 지워진 상태로 나옴 -> Google Colab에서 만드는 것을 추천*

In [ ]:
# load best model
load_path = os.getenv('HOME') + '/aiffel/mlops/best_model/1/model'
best_model = tf.keras.models.load_model(load_path)

In [ ]:
# 모델을 tflite 파일 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
# 변환한 tflite 파일 저장
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# 변환이 잘 되었는지 서명부분 확인
interpreter = tf.lite.Interpreter(model_content=tflite_model)

signatures = interpreter.get_signature_list()
print(signatures)

In [ ]:
# 서명 작동하는지 확인
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite